In [195]:
%pip install -r notebook_requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [196]:
import numpy as np
import pandas as pd
import os

#For Preprocessing
import re    # RegEx for removing non-letter characters

##  À partir du script ci-dessous, nous avons pu obtenir les recents tweets avec le #BITCOIN en anglais

## Créons un algorithme pour récupérer les récents tweets avec le hashtag bitcoin

### L'API twitter étant payante, nous avons du créer un compte et simuler le chargement de la recherche à partir du navigateur

In [197]:
def fetch_twitter():
    import http.client
    from dotenv import load_dotenv
    load_dotenv()
    conn = http.client.HTTPSConnection("twitter.com")

    headers = {
        'cookie': "des_opt_in=N; d_prefs=MjoxLGNvbnNlbnRfdmVyc2lvbjoyLHRleHRfdmVyc2lvbjoxMDAw; g_state=%7B%22i_p%22%3A1683828026348%2C%22i_l%22%3A1%7D; guest_id=v1%253A168382082645353816; _twitter_sess=BAh7BiIKZmxhc2hJQzonQWN0aW9uQ29udHJvbGxlcjo6Rmxhc2g6OkZsYXNo%25250ASGFzaHsABjoKQHVzZWR7AA%25253D%25253D--1164b91ac812d853b877e93ddb612b7471bebc74; kdt=7iW3OonvnZZrplohikZMqNcWdZgtGsFePCNqo9IC; auth_token=251d0278df8c7e41779f0269c79ae6c197f857d8; ct0=8e4fcb6328b604f7b95e629a4da02567e87db82f0b94ad331c4ecaf797942aea230b17d0c71bad6c51827094c11dcd2f5f45425dabdd42a24a795c0e18c90edb756723ffa0ccbed6f1b8443c79a0941e; lang=en; twid=u%253D1656691580143583236",
        'authority': "twitter.com",
        'accept': "*/*",
        'accept-language': "en,fr;q=0.9",
        'authorization': f"Bearer {os.getenv('BEARER_TOKEN')}",
        'dnt': "1",
        'referer': "https://twitter.com/search?q=-giving%20-givaway%20-give%20-win%20(%23bitcoin)%20min_faves%3A1000%20min_retweets%3A100%20lang%3Aen%20-filter%3Alinks%20-filter%3Areplies&src=typed_query",
        'sec-ch-ua': '"Chromium";v="113", "Not-A.Brand";v="24"',
        'sec-ch-ua-mobile': "?0",
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': "empty",
        'sec-fetch-mode': "cors",
        'sec-fetch-site': "same-origin",
        'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
        'x-client-uuid': "d4a81861-d1b3-4ea5-83b4-c32a71a3ea90",
        'x-csrf-token': "8e4fcb6328b604f7b95e629a4da02567e87db82f0b94ad331c4ecaf797942aea230b17d0c71bad6c51827094c11dcd2f5f45425dabdd42a24a795c0e18c90edb756723ffa0ccbed6f1b8443c79a0941e",
        'x-twitter-active-user': "yes",
        'x-twitter-auth-type': "OAuth2Session",
        'x-twitter-client-language': "en"
        }

    # Specific endpoint specifying : 
    # #bitcoin, 
    # 1000 minimum likes,
    # 100 minimum retweets,
    # not including (give, givaway, win, giving)
    conn.request("GET", "/i/api/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&include_ext_is_blue_verified=1&include_ext_verified_type=1&include_ext_profile_image_shape=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_ext_limited_action_results=false&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_ext_views=true&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=-giving%20-givaway%20-give%20-win%20(%23bitcoin)%20min_faves%3A1000%20min_retweets%3A100%20lang%3Aen%20-filter%3Alinks%20-filter%3Areplies&query_source=typed_query&count=20&requestContext=launch&pc=1&spelling_corrections=1&include_ext_edit_control=true&ext=mediaStats%2ChighlightedLabel%2ChasNftAvatar%2CvoiceInfo%2CbirdwatchPivot%2Cenrichments%2CsuperFollowMetadata%2CunmentionInfo%2CeditControl%2Cvibe", headers=headers)

    res = conn.getresponse()
    data = res.read()
    return data.decode("utf-8")

In [198]:
def parse_twitter_response(response):
    import json
    response = json.loads(response)
    return [
        {
            'text': response['globalObjects']['tweets'][tweet_id]['full_text'],
            'date': response['globalObjects']['tweets'][tweet_id]['created_at'],
        } 
        for tweet_id in response['globalObjects']['tweets']
    ]

In [199]:
def load_new_tweets(tweets):
    # Check if the fetched tweets are new
    # Load history tweets
    try:
        with open('new_tweets.json', 'r') as f:
            loaded_tweets = json.load(f)
            # Compare the new tweets with the previous ones
    except FileNotFoundError:
        print('First time fetching tweets')
    
    # Write new tweets to new_tweets.json
    with open('new_tweets.json', 'w',encoding='utf-8') as f:
        json.dump(tweets, f)

    # Load history tweets
    try:
        with open('history_tweets.json', 'r') as f:
            loaded_tweets = json.load(f)
    except FileNotFoundError:
        print('No history file found')

    # Merge new tweets with history tweets
    with open('history_tweets.json', 'w',encoding='utf-8') as f:
        json.dump(tweets + loaded_tweets, f)

In [200]:
tweets = parse_twitter_response(fetch_twitter())

## Chargeons le dictionnaire de Loughran et McDonald

In [201]:
LOUGHRAN_MCDONALD_DATASET = pd.read_csv('./dataset/Loughran-McDonald_MasterDictionary_1993-2021.csv')
LOUGHRAN_MCDONALD_DATASET 

,Word,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Syllables,Source
0,AARDVARK,1,354,1.550080e-08,1.422600e-08,3.815486e-06,99,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.313627e-10,8.653817e-12,9.241714e-09,1,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,9,3.940882e-10,1.169679e-10,5.290465e-08,7,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,29,1.269840e-09,6.654735e-10,1.595100e-07,28,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,8570,3.752595e-07,3.809464e-07,3.529356e-05,1108,0,0,0,0,0,0,0,3,12of12inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86526,ZYGOTE,86529,50,2.189379e-09,8.729336e-10,1.886011e-07,35,0,0,0,0,0,0,0,2,12of12inf
86527,ZYGOTES,86530,1,4.378757e-11,1.809516e-11,1.932446e-08,1,0,0,0,0,0,0,0,2,12of12inf
86528,ZYGOTIC,86531,0,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,3,12of12inf
86529,ZYMURGIES,86532,0,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,3,12of12inf


In [231]:
def data_cleaning(data : dict) -> list:
    """ Nous devons nettoyer le dataset pour ne garder que les mots qui nous intéressent
    Pour cela nous utilisons les stop words qui sont définis dans le fichier StopWords_Generix.txt
    """
    result = []
    with open('./dataset/StopWords_Generic.txt', 'r') as f:
        stop_words = f.read().split('\n')
    for element in data:
        for (header,content) in element.items():
            if header == 'text':
                modified = ' '.join([(''.join(letter.upper() for letter in word if letter.isalnum())) for word in content.split(' ')])
            else:
                date = content
        result.append( {
            'text': modified,
            'date':date
        })
    return result

data_cleaning(tweets)

[{'text': 'JUST IN BITCOIN MINER MARATHON PARTNERS WITH ZERO TWO BACKED BY ABU DHABIS 853 BILLION SOVEREIGN WEALTH FUND TO CREATE THE MIDDLE EASTS FIRST LARGE SCALE IMMERSION BTC MINING OPERATION ', 'date': 'Tue May 09 21:09:43 +0000 2023'}, {'text': 'FUN FACT 11 YEARS AGO TODAY THE BITCOIN PRICE WAS 502 ', 'date': 'Mon May 08 20:50:11 +0000 2023'}, {'text': 'MONG IS THE NEXT BITCOIN', 'date': 'Sat May 13 00:17:16 +0000 2023'}, {'text': ' BITCOIN HALTED ETHEREUM FEES TOO EXPENSIVE PULSECHAIN MAINNET LAUNCH', 'date': 'Wed May 10 12:47:32 +0000 2023'}, {'text': 'JUST IN  JUDGE RULES THAT BLOCKFI USERS GAVE UP LEGAL RIGHTS TO THEIR BTC BY USING THE PLATFORM AND ALL THE 300 MILLION OF CRYPTO DEPOSITS ARE NOW PROPERTY OF BLOCKFITAKE YOUR BITCOIN OFF EXCHANGES', 'date': 'Thu May 11 17:03:41 +0000 2023'}, {'text': 'IT REALLY HURTS MY FEELINGS THAT PEOPLE ARE BUYING BITCOIN AND WITHDRAWING MONEY FROM BANKSWHAT IS WRONG WITH PEOPLEI TRUST GARY GENSLER ELIZABETH WARREN NANCY PELOSI AND ALL OF TH

## Maintenant que le dictionnaire est chargé, nous pouvons créer notre algorithme pour calculer un score : positif ou negatif pour chaque texte